In [1]:
import json
import os
import sys
import random
from tqdm import tqdm
sys.path.append('..')
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
from typing import List, Union, Tuple
from rag_systems.retrieval.retrieval import Qdrant, KeywordMatching

# read documents from benchmarks/aurp/aurp-database/documents.json
root_dir = "spider/"
linguistic_control = "short"

## Gen

In [2]:
from grammar.generator import Generator
from grammar.llm import OpenAILLMAgent, role
gen_model = OpenAILLMAgent(model_name = "chatgpt-16k", role=role)
gen_model.temperature = 0
class LLMGen(Generator):
    verbalizer = {
        "short": "",
        "long": "",
        "concise": " The answer should be CONCISE"
    }
    def __init__(self, llm=None, verbalize_attrs=''):
        llm = llm or gen_model
        super().__init__( llm=gen_model, verbalize_attrs=verbalize_attrs)

    def _generate(self, query:str, num_generations=None, verbose=False):
        return [gen_model(query+ self.verbalizer["concise"], temperature=0)]

llm_gen = LLMGen.from_file(root_dir=root_dir, verbalize_attrs=linguistic_control)
# load evaluation data
file_path = f'{root_dir}/QADataGenerator/{linguistic_control}.json'
with open(file_path) as f:
    answers_to_text_queries = json.load(f)

# reduce the dimension over SQL templates 
semantics_groups = [(answer, query_list) for i in range(len(answers_to_text_queries)) for answer, query_list in answers_to_text_queries[i]]
print(len(semantics_groups))

57


In [41]:
results = [] 
group_tag = -1

for answer, query_list in semantics_groups:
    group_tag += 1 
    answer = eval(answer)[0]  # only 1 answer; no query multiplicity
    answer_txt = answer[0] 

    print(f"{len(query_list)} queries in group {group_tag}")

    for query in query_list:
        # if linguistic_control == "long":
        #     # 0.3 chance to select the example for eval; otherwise, skip
        #     if group_tag >= 38 and random.random() > 0.3:
        #         continue
        # else:
        #     # 0.75 chance to select the example for eval; otherwise, skip
        #     if random.random() > 0.75:
        #         continue
        
        result=  {"query": query, "answer": answer_txt}
        try:
            result["gpt_response"]  = llm_gen.generate(query)[0]
        except:
            result["gpt_response"] = None
            print(f"Query: {query}")
        
        result['query_tag'] = group_tag
        results.append(result)            


10 queries in group 0
10 queries in group 1
10 queries in group 2
10 queries in group 3
10 queries in group 4
10 queries in group 5
10 queries in group 6
10 queries in group 7
10 queries in group 8
10 queries in group 9
10 queries in group 10
10 queries in group 11
10 queries in group 12
10 queries in group 13
10 queries in group 14
10 queries in group 15
10 queries in group 16
10 queries in group 17
10 queries in group 18
10 queries in group 19
10 queries in group 20
10 queries in group 21
10 queries in group 22
10 queries in group 23
10 queries in group 24
10 queries in group 25
10 queries in group 26
10 queries in group 27
10 queries in group 28
10 queries in group 29
10 queries in group 30
10 queries in group 31
10 queries in group 32
10 queries in group 33
10 queries in group 34
10 queries in group 35
10 queries in group 36
10 queries in group 37
10 queries in group 38
10 queries in group 39
10 queries in group 40
10 queries in group 41
10 queries in group 42
10 queries in group 4

In [37]:
llm_gen.save(root_dir=root_dir, override=True)
gen_model.gpt_usage_record.write_usage(model_name='chatgpt16k' )

In [42]:
# save results
with open(f'{root_dir}/eval_results/gpt3.5-turbo_results_{linguistic_control}.json', 'w') as f:
    json.dump(results, f, indent=4)



## Eval

In [3]:
from grammar.eval.result import Result
from grammar.eval.match import SemanticsMatch

# read
with open(f'{root_dir}/eval_results/gpt3.5-turbo_results_{linguistic_control}.json', 'r') as f:
    results = json.load(f)
semnatics_match = SemanticsMatch.from_file(root_dir=root_dir, verbalize_attrs="gpt3.5-turbo_"+linguistic_control)

In [12]:
import time
eval_results = []
for result in tqdm(results):
    # sleep for 20 seconds after 9 examples
    if results.index(result)>10 and results.index(result) % 9 == 0 and results.index(result) != 0:
        print("Sleeping for 20 seconds")
        time.sleep(20)
        print("Waking up")
    eval_result = Result(**result)
    eval_result.judge_response(semnatics_match)
    eval_results.append(eval_result)

# save
semnatics_match.save(root_dir=root_dir, override=True)
semnatics_match.llm.gpt_usage_record.write_usage(model_name='chatgptk' )
# save results
with open(f'{root_dir}/eval_results/gpt3.5-turbo_results_{linguistic_control}.json', 'w') as f:
    json.dump([result.asdict() for result in eval_results], f, indent=4)


  0%|          | 0/570 [00:00<?, ?it/s]

The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
LLM is generating... for the key:  ("Retrieve the industry type of the company n

  2%|▏         | 11/570 [00:00<00:22, 24.84it/s]

LLM is generating... for the key:  ("Fetch the industry associated with the company 'JPMorgan Chase'.", 'Banking', 'Banking and Financial Services')
LLM is generating... for the key:  ("Obtain the industry for the company labelled as 'JPMorgan Chase'.", 'Banking', 'Banking and Financial Services')
LLM is generating... for the key:  ("Fetch industry from a business with the name 'JPMorgan Chase'.", 'Banking', 'Banking and Financial Services')


  2%|▏         | 14/570 [00:01<01:19,  6.96it/s]

...

100%|█████████▉| 568/570 [7:11:42<00:36, 18.30s/it]

LLM is generating... for the key:  ("Service years at 'Volkswagen Group' for 'Jerry Corcoran'", 4, "I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. My primary function is to provide information and answer questions to the best of my knowledge and abilities. If you have any concerns about privacy or data security, please let me know, and I will do my best to address them.")


100%|█████████▉| 569/570 [7:11:43<00:12, 12.94s/it]

LLM is generating... for the key:  ("Length of employment for 'Jerry Corcoran' at 'Volkswagen Group'", 4, "I'm sorry, but I don't have access to personal employment records.")


100%|██████████| 570/570 [7:11:43<00:00, 45.44s/it]


## Tagging

In [19]:
# read
with open(f'{root_dir}/eval_results/gpt3.5-turbo_results_{linguistic_control}.json', 'r') as f:
    results = json.load(f)
print(results[0])

{'query': "Retrieve the industry type of the company named 'ExxonMobil'.", 'answer': 'Oil and gas', 'gpt_response': 'Oil and Gas', 'query_tag': 0, 'judgement': 'Correct'}


In [22]:
from grammar.eval.tag_group import TaggedGroup
tagged_group = TaggedGroup([Result(**result) for result in results])
print(tagged_group.all_tags)
print(tagged_group.gap_tags)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56}
{40, 41, 42, 43, 44, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56}
